In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE248830"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE248830"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE248830.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE248830.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE248830.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Unlocking Molecular mechanisms and identifying druggable targets in matched-paired brain metastasis of Breast and Lung cancers "
!Series_summary	"Introduction: The incidence of brain metastases in cancer patients is increasing, with lung and breast cancer being the most common sources. Despite advancements in targeted therapies, the prognosis remains poor, highlighting the importance to investigate the underlying mechanisms in brain metastases. The aim of this study was to investigate the differences in the molecular mechanisms involved in brain metastasis of breast and lung cancers. In addition, we aimed to identify cancer lineage-specific druggable targets in the brain metastasis. Methods: To that aim, a cohort of 44 FFPE tissue samples, including 22 breast cancer and 22 lung adenocarcinoma (LUAD) and their matched-paired brain metastases were collected. Targeted gene expression profiles of primary tumors were compared to their matched-paired br

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this appears to be gene expression data
# The summary mentions "Targeted gene expression profiles" using nCounter PanCancer IO 360™ Panel
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait, we can use histology which contains breast cancer information
trait_row = 2  # histology
age_row = 0  # age at diagnosis
gender_row = 1  # Sex

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert breast cancer histology to binary (1 for breast cancer, 0 for others)"""
    if value is None or 'unknown' in value.lower():
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # If it's adenocarcinoma (lung cancer), it's not breast cancer
    if 'adenocaricnoma' in value.lower():
        return 0
    
    # If it contains any breast cancer markers, it's breast cancer
    if any(marker in value.lower() for marker in ['tnbc', 'er+', 'er-', 'pr+', 'pr-', 'her2+', 'her2-']):
        return 1
    
    return None

def convert_age(value):
    """Convert age to continuous numeric value"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() in ['n.a', 'n.a.', 'unknown']:
        return None
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{'GSM7920782': [1.0, 49.0, 0.0], 'GSM7920783': [1.0, 44.0, 0.0], 'GSM7920784': [nan, 41.0, 0.0], 'GSM7920785': [1.0, 40.0, 0.0], 'GSM7920786': [1.0, 48.0, 0.0], 'GSM7920787': [nan, 42.0, 0.0], 'GSM7920788': [1.0, 47.0, 0.0], 'GSM7920789': [1.0, 53.0, 0.0], 'GSM7920790': [1.0, 41.0, 0.0], 'GSM7920791': [1.0, 74.0, 0.0], 'GSM7920792': [1.0, 58.0, 0.0], 'GSM7920793': [1.0, 51.0, 0.0], 'GSM7920794': [1.0, 55.0, 0.0], 'GSM7920795': [nan, 46.0, 0.0], 'GSM7920796': [1.0, 46.0, 0.0], 'GSM7920797': [1.0, 48.0, 0.0], 'GSM7920798': [1.0, 44.0, 0.0], 'GSM7920799': [1.0, 49.0, 0.0], 'GSM7920800': [1.0, 59.0, 0.0], 'GSM7920801': [1.0, 50.0, 0.0], 'GSM7920802': [1.0, 74.0, 0.0], 'GSM7920803': [1.0, 46.0, 0.0], 'GSM7920804': [0.0, 40.0, 0.0], 'GSM7920805': [0.0, 57.0, 1.0], 'GSM7920806': [0.0, 60.0, 1.0], 'GSM7920807': [0.0, 55.0, 0.0], 'GSM7920808': [0.0, 69.0, 0.0], 'GSM7920809': [0.0, nan, 0.0], 'GSM7920810': [0.0, nan, 1.0], 'GSM7920811': [0.0, 57.0, 1.0], 'GSM79

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE248830/GSE248830_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE248830/GSE248830_series_matrix.txt.gz
Found the matrix table marker at line 58
Gene data shape: (754, 44)
First 20 gene/probe identifiers:
['A2M', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B', 'API5', 'APLNR', 'APOE', 'APOL6', 'AQP9']


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers from the output above:
# ['A2M', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1', 'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B', 'API5', 'APLNR', 'APOE', 'APOL6', 'AQP9']
# These appear to be standard human gene symbols rather than probe IDs or other identifiers
# For example: A2M (Alpha-2-Macroglobulin), AKT1 (AKT Serine/Threonine Kinase 1), and APOE (Apolipoprotein E) are well-known gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Breast Cancer subtype data available
# age_row = 2
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print(f"No trait data ({trait}) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
    )

Gene data shape before normalization: (754, 44)
Gene data shape after normalization: (746, 44)
Normalized gene data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE248830.csv
Extracted clinical data shape: (3, 44)
Preview of clinical data (first 5 samples):
               GSM7920782  GSM7920783  GSM7920784  GSM7920785  GSM7920786
Breast_Cancer         1.0         1.0         NaN         1.0         1.0
Age                  49.0        44.0        41.0        40.0        48.0
Gender                0.0         0.0         0.0         0.0         0.0
Clinical data saved to ../../output/preprocess/Breast_Cancer/clinical_data/GSE248830.csv
Gene data columns (first 5): ['GSM7920782', 'GSM7920783', 'GSM7920784', 'GSM7920785', 'GSM7920786']
Clinical data columns (first 5): ['GSM7920782', 'GSM7920783', 'GSM7920784', 'GSM7920785', 'GSM7920786']
Found 44 common samples between gene and clinical data
Initial linked data shape: (44, 749)
Preview of linked data (first 5 rows, first 5 col

For the feature 'Breast_Cancer', the least common label is '1.0' with 19 occurrences. This represents 46.34% of the dataset.
The distribution of the feature 'Breast_Cancer' in this dataset is fine.



Quartiles for 'Age':
  25%: 48.0
  50% (Median): 55.078947368421055
  75%: 60.0
Min: 37.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 12 occurrences. This represents 29.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Linked data saved to ../../output/preprocess/Breast_Cancer/GSE248830.csv
